<a href="https://colab.research.google.com/github/Jaydenzk/DS-repo/blob/master/Post_here_subreddit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import requests
import math

In [0]:
PAGE_COUNT = 5
RANDOM_STATE = 0
TEST_SIZE = 0.25

def get_reddit_data(page_count):
    print('fetching your data, * = 1 request to the Reddit API')
    headers = {'User-Agent': 'Predicting Reddit Post Metadata'}
    posts = []
    top_subreddits = [ 'r/funny', 'r/gaming', 'r/pics',
            'r/aww', 'r/science', 'r/worldnews', 'r/Music',
            'r/movies', 'r/todayilearned', 'r/videos'
    ]
    for i in range(len(top_subreddits)):
        after = ''
        for j in range(page_count):
            print('*', end='')
            url = 'https://www.reddit.com/'+ top_subreddits[i] + '/top.json?t=all&after=' + after
            #print(url)
            response = requests.get(url, headers=headers)    
            for k in range(len(response.json()['data']['children'])):
                post = {}
                post['created_utc'] = int(response.json()['data']['children'][k]['data']['created_utc'])
                post['is_video'] = int(response.json()['data']['children'][k]['data']['is_video'])
                post['subreddit'] = response.json()['data']['children'][k]['data']['subreddit']
                post['title'] = response.json()['data']['children'][k]['data']['title']
                post['total_awards_received'] = response.json()['data']['children'][k]['data']['total_awards_received']
                post['ups'] = response.json()['data']['children'][k]['data']['ups']
                posts.append(post)
            after = response.json()['data']['after']    
    return posts

columns = ['created_utc', 'is_video', 'subreddit', 'title', 'total_awards_received', 'ups']
df = pd.DataFrame(get_reddit_data(PAGE_COUNT), columns=columns)
df = df.sample(frac=1, random_state=RANDOM_STATE)

fetching your data, * = 1 request to the Reddit API
**************************************************

In [0]:
print('df.Describe():\n', df.describe(), '\n')
print(df.info(), '\n')
print('df.head(10):\n', df.head(10), '\n')
print('SUM OF NA VALUES:\n', df.isna().sum(), '\n')

df.Describe():
         created_utc     is_video  total_awards_received            ups
count  1.250000e+03  1250.000000             1250.00000    1250.000000
mean   1.536007e+09     0.058400                8.51600  111916.842400
std    2.822121e+07     0.234592               30.55932   41493.460995
min    1.407798e+09     0.000000                0.00000   35402.000000
25%    1.514150e+09     0.000000                0.00000   77574.000000
50%    1.539458e+09     0.000000                2.00000  111620.000000
75%    1.557992e+09     0.000000                7.00000  138394.250000
max    1.580818e+09     1.000000              521.00000  349288.000000 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1250 entries, 711 to 684
Data columns (total 6 columns):
created_utc              1250 non-null int64
is_video                 1250 non-null int64
subreddit                1250 non-null object
title                    1250 non-null object
total_awards_received    1250 non-null int64
ups      

In [0]:
df.head(25)

,created_utc,is_video,subreddit,title,total_awards_received,ups
711,1511202915,0,worldnews,Pope Francis denounces climate change deniers,1,98787
898,1563793522,0,movies,James Cameron congratulates Avengers: Endgame ...,2,96204
186,1541352571,0,gaming,Steve Jobs said it first,7,129252
867,1501167461,0,Music,Steve Earle says modern country stars make 'hi...,0,36524
18,1548784436,0,funny,"Citizen of Golden, CO (home of Coors and about...",6,170710
1152,1562593061,0,videos,Let's not forget about the teacher who was arr...,6,101417
192,1529623801,0,gaming,"Youtuber ""Potates"" completely plagiarized a po...",10,127316
184,1479762343,0,gaming,Some Michael Bay shit going on right here,0,129561
824,1551698540,0,Music,Prodigy frontman Keith Flint found dead at his...,3,44634
1058,1519147317,0,todayilearned,TIL there's a restaurant in New York that does...,1,105334


In [0]:
from scipy.stats import zscore
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import make_pipeline

In [0]:
X = df['title'].values
y = df['subreddit'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=0)

In [0]:
model = make_pipeline(TfidfVectorizer(), MultinomialNB())
model.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('tfidfvectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('multinomialnb',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [0]:
y_predict = model.predict(X_test)
accuracy = (y_predict == y_test).mean()
baseline = pd.Series(y_test).value_counts()[0] / pd.Series(y_test).value_counts().sum()
prediction_count = pd.Series(y_test).value_counts().sum()

In [0]:
print('VC of y_train')
print(pd.Series(y_train).value_counts()[:10], '\n')
print('VC of y_test')
print(pd.Series(y_test).value_counts()[:10], '\n')
print('VC of y_predict')
print(pd.Series(y_predict).value_counts()[:10], '\n')
print('# PRDCTN: ', prediction_count)
print('BASELINE: ', baseline)
print('ACCURACY: ', accuracy)
print(len(y_test), len(y_predict))       

VC of y_train
pics             99
movies           99
Music            97
videos           97
gaming           95
worldnews        94
aww              92
todayilearned    91
science          88
funny            85
dtype: int64 

VC of y_test
funny            40
science          37
todayilearned    34
aww              33
worldnews        31
gaming           30
Music            28
videos           28
pics             26
movies           26
dtype: int64 

VC of y_predict
todayilearned    75
pics             49
aww              38
movies           37
science          34
videos           24
Music            18
worldnews        18
gaming           16
funny             4
dtype: int64 

# PRDCTN:  313
BASELINE:  0.12779552715654952
ACCURACY:  0.5207667731629393
313 313


In [0]:
train_predict_report(df[0:10000])

In [0]:
import pickle

In [0]:
pickle.dump(model, open( "model.pkl", "wb" ) )